# `swift_too` module

## Swift_PlanQuery example - querying planned *Swift* observations

### API version = 1.2

#### Author: Jamie A. Kennea (Penn State)

The Swift_PlanQuery class allows for querying the database of observations have have been planned for observation by Swift, otherwise known as the "Pre-Planned Science Timeline" (PPST). This allows the user to find out not only what was planned, but what will be observed by Swift in the future.

Two important caveats:

1. For past observations, what should have been down is mostly an academic exercise, as due to TOO uploads and GRBs, Swift's pre-planned observations can be obliterated. If you want to know what Swift *did* observe, do a Swift_ObsQuery

2. Swift's planning is only done days into the future, so if you want to know what is going to happen weeks from now, you're out of luck.

This therefore is mostly useful if you want to know what Swift is observing *today* and *tomorrow*.

In [1]:
from swifttools.swift_too import Swift_PlanQuery
from datetime import datetime,timedelta

### Setting up our username and secret key

Set up the class, giving our username and shared secret. These can be set up [on the Swift TOO website](https://www.swift.psu.edu/toop/too.php). After you log in, you will find your shared secret, and can modify it as necessary under the [Update Account Info](https://www.swift.psu.edu/toop/account_info.php) link. 

In [2]:
username = 'myuser'
shared_secret = 'mysharedsecret'

However, if you don't have a user or just don't want to use it, you can submit this request anonymously. In this case you just give the username and shared secret as "anonymous"

In [3]:
username = 'anonymous'
shared_secret = 'anonymous'

### First Query

First query example: What is Swift planned to observe now? Quite easy to construct. Here it is:

In [4]:
plan = Swift_PlanQuery(username,shared_secret,begin=datetime.utcnow())
plan

Begin,End,Name,Obs Number,Exposure (s)
2021-11-14 14:57:00,2021-11-14 15:22:00,Tycho_200_200,00059158044,1500


So the above should have reported back a single target. When you just give `start` with no `end`, it simply returns the observation which is taking place at the time given. So what is happening at this time tomorrow?

In [5]:
plan = Swift_PlanQuery(username,shared_secret,begin=datetime.utcnow()+timedelta(days=1))
plan

Begin,End,Name,Obs Number,Exposure (s)
2021-11-15 14:43:00,2021-11-15 15:02:00,OGP_2008,03103668007,1140


Did the above query return anything? If not, then there simply isn't a plan for tomorrow. So how can we determine how far into the future Swift has a plan for? The `ppstmax` parameter. Let's check it out:

In [6]:
f"PPSTs available up to {plan.ppstmax}"

'PPSTs available up to 2021-11-16 00:00:00'

This gives the date of the last planned target. If the query for what Swift is observing this time tomorrow didn't return anything, this date is probably the end of the UT day today. If the time you're running this is before midday on a Monday-Thursday, this is as expected. Typically the plan for tomorrow is submitted for upload around 12pm Eastern time. The exception to this rule is for the weekend plan. Typically on Thursday at 12pm we submit a plan for Friday and Saturday, and on Friday at 12pm we submit a plan for Sunday-Monday. This allows us to cover the weekend, while working a 5 day week.

### Second query example

Seeing a whole day's PPST. So let's see the whole day's Plan that goes farthest into the future.

In [7]:
dayplan = Swift_PlanQuery(username,shared_secret,begin=plan.ppstmax-timedelta(days=1),end=plan.ppstmax)
dayplan

Begin,End,Name,Obs Number,Exposure (s)
2021-11-14 23:47:00,2021-11-15 00:02:00,4FGL J1840.4-1139,03109925005,900
2021-11-15 00:02:00,2021-11-15 00:17:00,SGWGS-2856,03107647009,900
2021-11-15 00:17:00,2021-11-15 00:31:00,saa-cold-357-08,00075098022,840
2021-11-15 00:31:00,2021-11-15 00:43:00,IC 10 X-1,00030835119,720
2021-11-15 00:43:00,2021-11-15 00:57:00,1RXS_J065558.4+290731,00049293004,840
2021-11-15 00:57:00,2021-11-15 01:27:00,MRK 477,00095968009,1800
2021-11-15 01:27:00,2021-11-15 01:36:00,4FGL J1853.8-1107,03110350007,540
2021-11-15 01:36:00,2021-11-15 01:42:00,PSR J1835-1106,00036698006,360
2021-11-15 01:42:00,2021-11-15 02:00:00,SGWGS-2397,03107188004,1080
2021-11-15 02:00:00,2021-11-15 02:26:00,MRK 590,00095662051,1560


That's a lot of targets! Note that like Swift_ObsQuery it's showing you all the individual snapshots during the day. If you want to group these into observations, you can do that. Remember the `observations` method returns a dictionary, with obsid as the key, and the total observation details as the value. You can display it as a table in Jupyter Notebooks simply:

In [8]:
dayplan.observations

Begin,End,Name,Obs Number,Exposure (s),Slewtime (s)
2021-11-14 23:47:00,2021-11-15 00:02:00,4FGL J1840.4-1139,03109925005,900,0
2021-11-15 00:02:00,2021-11-15 00:17:00,SGWGS-2856,03107647009,900,0
2021-11-15 00:17:00,2021-11-15 12:47:00,saa-cold-357-08,00075098022,1260,0
2021-11-15 00:31:00,2021-11-15 00:43:00,IC 10 X-1,00030835119,720,0
2021-11-15 00:43:00,2021-11-15 00:57:00,1RXS_J065558.4+290731,00049293004,840,0
2021-11-15 00:57:00,2021-11-15 20:34:00,MRK 477,00095968009,9840,0
2021-11-15 01:27:00,2021-11-15 01:36:00,4FGL J1853.8-1107,03110350007,540,0
2021-11-15 01:36:00,2021-11-15 01:42:00,PSR J1835-1106,00036698006,360,0
2021-11-15 01:42:00,2021-11-15 02:00:00,SGWGS-2397,03107188004,1080,0
2021-11-15 02:00:00,2021-11-15 05:21:00,MRK 590,00095662051,2220,0
